In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine
from ling_features import word_count, number_count, sent_count

conn_string = 'postgresql://localhost/crsp'

# Doesn't need to expand_json for this one
"""def expand_json(df, col):
    return pd.concat([df.drop([col], axis=1),
                      df[col].map(lambda x: json.loads(json.dumps(x))).apply(pd.Series)], axis=1)"""

engine = create_engine(conn_string)
        
sql =  """
SELECT file_name, last_update, context, section,
   string_agg(speaker_text, ' ') AS speaker_text
FROM streetevents.speaker_data
WHERE file_name IN ('13113375_T', '13132191_T', '13168304_T', '13135070_T', '13135529_T')
GROUP BY file_name, last_update, context, section
ORDER BY file_name, last_update, context, section"""

speaker_data = pd.read_sql(sql, engine)
speaker_data['last_update'] = speaker_data['last_update'].map(lambda x: str(x.astimezone('UTC')))

In [2]:
speaker_data['sum'] = speaker_data['speaker_text'].apply(word_count)
speaker_data['sent_count'] = speaker_data['speaker_text'].apply(sent_count)
speaker_data['sum_6'] = speaker_data['speaker_text'].apply(word_count, args = (6,))
speaker_data['sum_num'] = speaker_data['speaker_text'].apply(number_count)
speaker_data = speaker_data.drop(['speaker_text'], axis=1)

In [3]:
speaker_data

,file_name,last_update,context,section,sum,sent_count,sum_6,sum_num
0,13113375_T,2020-06-11 17:14:37+00:00,pres,1,3082,138,1055,78
1,13113375_T,2020-06-11 17:14:37+00:00,qa,1,4055,225,953,10
2,13132191_T,2020-05-06 06:26:34+00:00,pres,1,5049,188,1692,85
3,13132191_T,2020-05-06 06:26:34+00:00,qa,1,5693,286,1287,27
4,13135070_T,2020-06-12 12:53:14+00:00,pres,1,1852,85,547,65
5,13135070_T,2020-06-12 12:53:14+00:00,qa,1,3633,239,673,29
6,13135529_T,2020-06-29 17:13:22+00:00,pres,1,1476,55,525,19
7,13135529_T,2020-06-29 17:13:22+00:00,qa,1,5203,243,1189,26
8,13168304_T,2020-05-07 05:38:25+00:00,pres,1,3608,154,1356,65
9,13168304_T,2020-05-07 05:38:25+00:00,qa,1,6558,321,1472,22


In [4]:
word_count("This is Ian's sample sentence.")

7